<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# Wide and Deep Model for Movie Recommendation
<br>

This notebook shows how to build and test [**wide-and-deep model**](https://arxiv.org/abs/1606.07792)--linear combination of linear and deep NN models--using [TensorFlow high-level Estimator API](https://www.tensorflow.org/api_docs/python/tf/estimator/DNNLinearCombinedRegressor).

On the [movie recommendation dataset](https://grouplens.org/datasets/movielens/), we quickly demonstrate how to prepare features, build the model, use log-hook to estimate performance while training, export model, and load the saved model.

We use a preset of hyper-parameters optimized for *MovieLens 1M* dataset we found by utilizing **Azure Machine Learning service**([AzureML or AML](https://azure.microsoft.com/en-us/services/machine-learning-service/)). You can find more details about hyperparameter tuning of wide-and-deep model via AML from [our notebook](../04_model_select_and_optimize/aml_hyperparameter_tuning.ipynb).

### Prerequisite
* TensorFlow (version 1.8 or higher) - GPU version is preferable. To easily setup a conda environment with `tensorflow-gpu` package, please follow [SETUP].(https://github.com/Microsoft/Recommenders/blob/master/SETUP.md) 

In [1]:
import sys
sys.path.append("../../")

import os
import shutil
import itertools

import tensorflow as tf
import pandas as pd
import numpy as np
import sklearn.preprocessing

from reco_utils.common import tf_utils
from reco_utils.dataset import movielens
from reco_utils.dataset.pandas_df_utils import user_item_pairs
from reco_utils.dataset.python_splitters import python_random_split
from reco_utils.evaluation.python_evaluation import (
    rmse, mae, rsquared, exp_var,
    map_at_k, ndcg_at_k, precision_at_k, recall_at_k
)

In [2]:
from tensorflow.python.client import device_lib

print("Tensorflow Version:", tf.__version__)

devices = device_lib.list_local_devices()
[x.name for x in devices]

Tensorflow Version: 1.12.0


['/device:CPU:0', '/device:XLA_CPU:0', '/device:XLA_GPU:0', '/device:GPU:0']

### Data loading

Download [MovieLens](https://grouplens.org/datasets/movielens/) data and split train / test set.

MovieLens data have movie genres information where each movie has one or more genres. We do multi-hot-encode to use genres as an item feature. For example:

*Movie id=2355* has three genres, *Animation | Children's | Comedy*, which are being converted into an integer array of the indicator value for each genre like `[0, 0, 1, 1, 1, 0, 0, 0, ...]`.

In [3]:
# top k items to recommend
TOP_K = 10

# Select Movielens data size: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = '1m'

USER_COL = 'UserId'
ITEM_COL = 'MovieId'
RATING_COL = 'Rating'
ITEM_FEAT_COL = 'Genres'

In [4]:
data = movielens.load_pandas_df(
    size=MOVIELENS_DATA_SIZE,
    header=[USER_COL, ITEM_COL, RATING_COL],
    genres_col='Genres_string'
)
data.head()

,UserId,MovieId,Rating,Genres_string
0,1,1193,5.0,Drama
1,2,1193,5.0,Drama
2,12,1193,4.0,Drama
3,15,1193,4.0,Drama
4,17,1193,5.0,Drama


In [5]:
# Encode 'genres' into int array (multi-hot representation) to use as item features
genres_encoder = sklearn.preprocessing.MultiLabelBinarizer()
data[ITEM_FEAT_COL] = genres_encoder.fit_transform(
    data['Genres_string'].apply(lambda s: s.split("|"))
).tolist()
print("Genres:", genres_encoder.classes_, "\n")

_items = data.drop_duplicates(ITEM_COL)[[ITEM_COL, 'Genres_string', ITEM_FEAT_COL]].reset_index(drop=True)
print(_items.head())

Genres: ['Action' 'Adventure' 'Animation' "Children's" 'Comedy' 'Crime'
 'Documentary' 'Drama' 'Fantasy' 'Film-Noir' 'Horror' 'Musical' 'Mystery'
 'Romance' 'Sci-Fi' 'Thriller' 'War' 'Western'] 

   MovieId                 Genres_string  \
0     1193                         Drama   
1      661  Animation|Children's|Musical   
2      914               Musical|Romance   
3     3408                         Drama   
4     2355   Animation|Children's|Comedy   

                                              Genres  
0  [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...  
1  [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...  
2  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, ...  
3  [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...  
4  [0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  


In [6]:
# Unique items and users in the data. Will be used to generate recommendation pool
items = _items.drop('Genres_string', axis=1)
users = data.drop_duplicates(USER_COL)[[USER_COL]].reset_index(drop=True)

print("num items = {}, num users = {}".format(len(items), len(users)))

num items = 3706, num users = 6040


In [7]:
train, test = python_random_split(
    data.drop('Genres_string', axis=1),
    ratio=0.75,
    seed=123
)

### Modeling

Wide and deep model utilizes two different types of feature sets:
1. Wide / cross-producted features to capture how the co-occurrence of a user-item feature pair correlates with the target label or rating, and
2. Deep / lower-dimensional embedding vectors for every user and item.

In [9]:
""" Hyper parameters
"""
BATCH_SIZE = 256
NUM_EPOCHS = int(20000000 / len(train))
print("Num epochs: ", NUM_EPOCHS)

LINEAR_OPTIMIZER = tf.train.FtrlOptimizer(
    learning_rate=0.07,
    l1_regularization_strength=0.015
)
DNN_OPTIMIZER = tf.train.AdagradOptimizer(
    learning_rate=0.018
)
DNN_HIDDEN_UNITS = [64, 128, 32]
DNN_DROPOUT = 0.2
DNN_BATCH_NORM = True

DNN_USER_DIM = 32
DNN_ITEM_DIM = 16

print("Embedding {} users to {}-dim vector".format(len(users), DNN_USER_DIM))
print("Embedding {} items to {}-dim vector".format(len(items), DNN_ITEM_DIM))

Num epochs:  26
Embedding 6040 users to 32-dim vector
Embedding 3706 items to 16-dim vector


In [10]:
""" Feature columns
"""

user_id = tf.feature_column.categorical_column_with_vocabulary_list(USER_COL, users[USER_COL].values)
item_id = tf.feature_column.categorical_column_with_vocabulary_list(ITEM_COL, items[ITEM_COL].values)

wide_columns = [
    tf.feature_column.crossed_column([user_id, item_id], hash_bucket_size=1000)
]

deep_columns = [
    # User embedding
    tf.feature_column.embedding_column(
        categorical_column=user_id,
        dimension=DNN_USER_DIM,
        max_norm=DNN_USER_DIM ** .5
    ),
    # Item embedding
    tf.feature_column.embedding_column(
        categorical_column=item_id,
        dimension=DNN_ITEM_DIM,
        max_norm=DNN_ITEM_DIM ** .5
    ),
    # Item feature
    tf.feature_column.numeric_column(
        ITEM_FEAT_COL,
        shape=len(genres_encoder.classes_),
        dtype=tf.float32
    )
]

for c in wide_columns + deep_columns:
    print(str(c)[:100], "...")

_CrossedColumn(keys=(_VocabularyListCategoricalColumn(key='UserId', vocabulary_list=(1, 2, 12, 15, 1 ...
_EmbeddingColumn(categorical_column=_VocabularyListCategoricalColumn(key='UserId', vocabulary_list=( ...
_EmbeddingColumn(categorical_column=_VocabularyListCategoricalColumn(key='MovieId', vocabulary_list= ...
_NumericColumn(key='Genres', shape=(18,), default_value=None, dtype=tf.float32, normalizer_fn=None) ...


In [11]:
MODEL_DIR = 'model_checkpoints'

try:
    # Clean-up previous model dir if exists
    shutil.rmtree(MODEL_DIR)
except (PermissionError, FileNotFoundError):
    pass

# Set logging frequency
LOG_STEP = 1000
run_config = tf.estimator.RunConfig()
run_config = run_config.replace(log_step_count_steps=LOG_STEP)

# We use regressor for rating prediction
model = tf.estimator.DNNLinearCombinedRegressor(
    model_dir=MODEL_DIR,
    config=run_config,
    # wide model args
    linear_feature_columns=wide_columns,
    linear_optimizer=LINEAR_OPTIMIZER,
    # deep model args
    dnn_feature_columns=deep_columns,
    dnn_hidden_units=DNN_HIDDEN_UNITS,
    dnn_optimizer=DNN_OPTIMIZER,
    dnn_dropout=DNN_DROPOUT,
    batch_norm=DNN_BATCH_NORM
)

INFO:tensorflow:Using config: {'_model_dir': 'model_checkpoints', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 1000, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f17f1a266d8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [12]:
# Add additional metrics 'mae' in addition to the default 'loss'
metrics_fn = (lambda labels, predictions: {
    'mae': tf.metrics.mean_absolute_error(
        tf.cast(labels, tf.float32),
        predictions['predictions']
    )
})

model = tf.contrib.estimator.add_metrics(model, metrics_fn)

INFO:tensorflow:Using config: {'_model_dir': 'model_checkpoints', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 1000, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f17bcddd518>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


### Training and Evaluation

In [13]:
# Prepare a recommendation pool for recommending k-item (ranking) scenario
# We also remove seen items (filter out user-item pairs in the training set)
ranking_pool = user_item_pairs(
    user_df=users,
    item_df=items,
    user_col=USER_COL,
    item_col=ITEM_COL,
    user_item_filter_df=train,
    shuffle=True
)

ranking_pool.head()

,UserId,MovieId,Genres
0,3479,22,"[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
1,4215,218,"[0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
2,3528,1652,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,646,1078,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,5105,2834,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."


In [14]:
""" Training hooks
"""
eval_kwargs = {
    'col_user': USER_COL,
    'col_item': ITEM_COL,
    'col_rating': RATING_COL,
    'col_prediction': 'prediction',
    'k': TOP_K
}

precision_eval_hook = tf_utils.evaluation_log_hook(
    model,
    true_df=test,
    y_col=RATING_COL,
    eval_df=ranking_pool,
    every_n_iter=LOG_STEP*20,
    model_dir=MODEL_DIR,
    eval_fn=precision_at_k,
    **eval_kwargs
)

In [15]:
model.train(
    input_fn=tf_utils.pandas_input_fn(
        df=train,
        y_col=RATING_COL,
        batch_size=BATCH_SIZE,
        num_epochs=NUM_EPOCHS,
        shuffle=True
    ),
    hooks=[precision_eval_hook]
)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into model_checkpoints/model.ckpt.
INFO:tensorflow:loss = 4215.095, step = 0
INFO:tensorflow:global_step/sec: 109.255
INFO:tensorflow:loss = 315.16928, step = 1000 (9.154 sec)
INFO:tensorflow:global_step/sec: 110.431
INFO:tensorflow:loss = 255.80698, step = 2000 (9.055 sec)
INFO:tensorflow:global_step/sec: 116.352
INFO:tensorflow:loss = 268.7221, step = 3000 (8.595 sec)
INFO:tensorflow:global_step

INFO:tensorflow:global_step/sec: 115.88
INFO:tensorflow:loss = 220.82213, step = 44000 (8.630 sec)
INFO:tensorflow:global_step/sec: 116.156
INFO:tensorflow:loss = 168.3501, step = 45000 (8.609 sec)
INFO:tensorflow:global_step/sec: 113.036
INFO:tensorflow:loss = 214.42038, step = 46000 (8.848 sec)
INFO:tensorflow:global_step/sec: 115.794
INFO:tensorflow:loss = 218.62462, step = 47000 (8.635 sec)
INFO:tensorflow:global_step/sec: 116.867
INFO:tensorflow:loss = 207.54907, step = 48000 (8.557 sec)
INFO:tensorflow:global_step/sec: 112.824
INFO:tensorflow:loss = 154.18506, step = 49000 (8.863 sec)
INFO:tensorflow:global_step/sec: 116.404
INFO:tensorflow:loss = 200.54301, step = 50000 (8.591 sec)
INFO:tensorflow:global_step/sec: 117.022
INFO:tensorflow:loss = 165.60431, step = 51000 (8.547 sec)
INFO:tensorflow:global_step/sec: 110.745
INFO:tensorflow:loss = 209.86362, step = 52000 (9.028 sec)
INFO:tensorflow:global_step/sec: 117.687
INFO:tensorflow:loss = 198.30777, step = 53000 (8.497 sec)
IN

### Testing

We predict the ratings by using the wide-deep model we trained. Finally, we also generate top-k movie recommentation for each user and test the performance.

1. Item rating prediction

In [16]:
cols = {
    'col_user': USER_COL,
    'col_item': ITEM_COL,
    'col_rating': RATING_COL,
    'col_prediction': 'prediction'
}

predictions = list(model.predict(input_fn=tf_utils.pandas_input_fn(df=test)))
prediction_df = test.drop(RATING_COL, axis=1)
prediction_df['prediction'] = [p['predictions'][0] for p in predictions]
prediction_df['prediction'].describe()

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_checkpoints/model.ckpt-76188
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


count    250053.000000
mean          3.583720
std           0.652448
min          -0.539153
25%           3.167789
50%           3.703695
75%           4.079504
max           5.550684
Name: prediction, dtype: float64

In [17]:
eval_rmse = rmse(test, prediction_df, **cols)
eval_mae = mae(test, prediction_df, **cols)
eval_rsquared = rsquared(test, prediction_df, **cols)
eval_exp_var = exp_var(test, prediction_df, **cols)

print("RMSE:\t\t%f" % eval_rmse,
      "MAE:\t\t%f" % eval_mae,
      "rsquared:\t%f" % eval_rsquared,
      "exp var:\t%f" % eval_exp_var, sep='\n')

RMSE:		0.892138
MAE:		0.702158
rsquared:	0.360975
exp var:	0.360982


In [18]:
eval_results = model.evaluate(
    input_fn=tf_utils.pandas_input_fn(
        df=test,
        y_col=RATING_COL
    ),
    steps=None
)

print(eval_results)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-01-25-05:58:07
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_checkpoints/model.ckpt-76188
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-01-25-05:58:16
INFO:tensorflow:Saving dict for global step 76188: average_loss = 0.7959096, global_step = 76188, label/mean = 3.580709, loss = 101.852394, mae = 0.7021575, prediction/mean = 3.5837185
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 76188: model_checkpoints/model.ckpt-76188
{'average_loss': 0.7959096, 'label/mean': 3.580709, 'loss': 101.852394, 'mae': 0.7021575, 'prediction/mean': 3.5837185, 'global_step': 76188}


2. Recommend k items

In [19]:
predictions = list(model.predict(input_fn=tf_utils.pandas_input_fn(df=ranking_pool)))
prediction_df = ranking_pool.copy()
prediction_df['prediction'] = [p['predictions'][0] for p in predictions]

eval_map = map_at_k(test, prediction_df, k=TOP_K, **cols)
eval_ndcg = ndcg_at_k(test, prediction_df, k=TOP_K, **cols)
eval_precision = precision_at_k(test, prediction_df, k=TOP_K, **cols)
eval_recall = recall_at_k(test, prediction_df, k=TOP_K, **cols)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_checkpoints/model.ckpt-76188
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
MAP:	0.005812
NDCG:	0.059414
Precision@K:	0.060116
Recall@K:	0.018249


### Tensorboard

To see Tensorboard, open a terminal from this notebook folder, run `tensorboard --logdir=model_checkpoints`, and open http://localhost:6006 from a browser.

### Export Model

In [20]:
EXPORT_DIR_BASE = 'saved_model'
os.makedirs(EXPORT_DIR_BASE, exist_ok=True)

In [21]:
train_rcvr_fn = tf.contrib.estimator.build_supervised_input_receiver_fn_from_input_fn(
    tf_utils.pandas_input_fn(
        df=train,
        y_col=RATING_COL,
        batch_size=BATCH_SIZE,
        num_epochs=NUM_EPOCHS,
        shuffle=True
    )
)
eval_rcvr_fn = tf.contrib.estimator.build_supervised_input_receiver_fn_from_input_fn(
    tf_utils.pandas_input_fn(
        df=test,
        y_col=RATING_COL
    )
)
serve_rcvr_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(
    tf.feature_column.make_parse_example_spec(wide_columns+deep_columns)
)
rcvr_fn_map = {
    tf.estimator.ModeKeys.TRAIN: train_rcvr_fn,
    tf.estimator.ModeKeys.EVAL: eval_rcvr_fn,
    tf.estimator.ModeKeys.PREDICT: serve_rcvr_fn
}

export_dir = tf.contrib.estimator.export_all_saved_models(
    model,
    export_dir_base=EXPORT_DIR_BASE,
    input_receiver_fn_map=rcvr_fn_map
)

print("Model exported to", export_dir)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: None
INFO:tensorflow:Signatures INCLUDED in export for Train: ['train']
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from model_checkpoints/model.ckpt-76188
Instructions for updating:
Pass your op to the equivalent parameter main_op instead.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signa

In [22]:
saved_model = tf.contrib.estimator.SavedModelEstimator(export_dir)

result = saved_model.evaluate(
    tf_utils.pandas_input_fn(
        df=test,
        y_col=RATING_COL
    ),
    steps=None
)
print(result)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpdsko823z', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f178b4a0198>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Checking available modes for SavedModelEstimator.
INFO:tensorflow:Available modes for Estimator: ['train', 'eval', 'infer']
I

In [23]:
test_sample = test.iloc[0]
test_sample

UserId                                                  4790
MovieId                                                 3481
Rating                                                     4
Genres     [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: 382276, dtype: object

In [24]:
def predict_input_fn():
    example = tf.train.Example()
    
    example.features.feature[USER_COL].int64_list.value.extend([test_sample[USER_COL]])
    example.features.feature[ITEM_COL].int64_list.value.extend([test_sample[ITEM_COL]])
    example.features.feature[ITEM_FEAT_COL].float_list.value.extend(test_sample[ITEM_FEAT_COL])
    return {'inputs':tf.constant([example.SerializeToString()])}

print(next(saved_model.predict(predict_input_fn)))

INFO:tensorflow:Could not find trained model in model_dir: /tmp/tmpdsko823z, running initialization to predict.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='saved_model/1548397777/variables/variables', vars_to_warm_start=['dnn/hiddenlayer_0/batchnorm_0/beta', 'dnn/hiddenlayer_0/batchnorm_0/beta/Adagrad', 'dnn/hiddenlayer_0/batchnorm_0/gamma', 'dnn/hiddenlayer_0/batchnorm_0/gamma/Adagrad', 'dnn/hiddenlayer_0/batchnorm_0/moving_mean', 'dnn/hiddenlayer_0/batchnorm_0/moving_variance', 'dnn/hiddenlayer_0/bias', 'dnn/hiddenlayer_0/bias/Adagrad', 'dnn/hiddenlayer_0/kernel', 'dnn/hiddenlayer_0/kernel/Adagrad', 'dnn/hiddenlayer_1/batchnorm_1/beta', 'dnn/hiddenlayer_1/batchnorm_1/beta/Adagrad', 'dnn/hiddenlayer_1/batchnorm_1/gamma', 'dnn/hiddenlayer_1/batchnorm_1/gamma/Adagrad', 'dnn/hiddenlayer_1/batchnorm_1/moving_mean', 'dnn/hiddenlayer_1/batchnorm_1/moving_variance', '

In [25]:
# Cleanup
shutil.rmtree(EXPORT_DIR_BASE)